In [ ]:
from annotate_utils import *

In [2]:
df = load_data()
df_filtered = filter_data(df)

loaded 60265 rows
filtered to 42354 rows


In [3]:
df_filtered.groupby('subreddit')["post_id"].count().head(12)

subreddit
Infidelity               844
LifeAfterNarcissism     2436
NarcAbuseAndDivorce     2032
NarcissisticSpouses     3593
ToxicRelationships      1099
abusesurvivors          2526
abusiverelationships    6087
domesticviolence        4059
emotionalabuse          3499
relationship_advice     7119
relationships           6181
survivinginfidelity     2878
Name: post_id, dtype: int64

#### Load the labeled data

In [4]:
import pickle
import pandas as pd 
labeled_metadata = pickle.load(open('/data/home/ilanit.sobol/dv/data/outputs/reddit/labeled/iter2/results_posts_labels.pkl', 'rb'))

In [ ]:
#### Join the results with the original data
results_df_labels = pd.DataFrame(labeled_metadata)
results_df_labels = results_df_labels[results_df_labels["output_json"] != None]
expanded_json_df_labels = pd.json_normalize(results_df_labels['output_json'])
expanded_json_df_labels["post_id"] = results_df_labels["post_id"]
expanded_json_df_labels = expanded_json_df_labels.set_index("post_id")
results_df_labels = results_df_labels.set_index("post_id")
results_df_joined_labels = results_df_labels.join(expanded_json_df_labels, how='inner')
results_df_joined_labels = results_df_joined_labels.drop(["story", "output_json"], axis=1)
final_df_joined_all = df_filtered.merge(results_df_joined_labels, left_on = "post_id", right_on="post_id", how='inner').reset_index()

In [11]:
final_df_joined_all["is_romantic"].value_counts()

yes                   21525
no                     2605
cannot be inferred      737
irrelevant               63
Name: is_romantic, dtype: int64

In [15]:
final_df_joined_all["relationship_type"].value_counts()

dating                        6549
married                       6480
relationship                  5924
family                        1834
other                         1486
cannot be inferred            1167
engaged                        681
friends                        615
co-workers                     117
roommates                       55
classmates                      17
situationship                    2
acquaintance of the victim       1
neighbors                        1
friends with benefits            1
Name: relationship_type, dtype: int64

In [21]:
final_df_joined_all.sample(3)

,post_id,title,post_author,body,date_time,url,score,num_comments,flairs,highest_comment_id,...,children,relationship_type,was_breakup,when_breakup,story_type,author_role,living_with,age_female,age_male,author_gender
18070,1c91uo7,Bf of 7 years who is 27M is too busy for me 27...,Embarrassed-Carob178,"Hi everyone,\n\nSo, my boyfriend and I have be...",20/04/24 22:04:58,https://www.reddit.com/r/relationships/comment...,2,1,[],l0izhvv,...,cannot be inferred,dating,no,irrelevant,advice request (without a personal story),other,cannot be inferred,27.0,27.0,female
10966,181v4uz,Can you be abused by a sibling?,ljxbb,I’m trying to figure this out. My brother is a...,23/11/23 06:47:00,https://www.reddit.com/r/abusesurvivors/commen...,8,19,[],kakko09,...,irrelevant,family,no,irrelevant,sharing a personal story,victim,cannot be inferred,NaN,NaN,female
7705,18abwhj,I feel like a I can't anymore.,Wildoves,This year has been so shitty. My grandfather ...,04/12/23 04:06:30,https://www.reddit.com/r/abusesurvivors/commen...,0,2,[],kbxcwk8,...,irrelevant,other,no,irrelevant,sharing a personal story,victim,cannot be inferred,NaN,NaN,female


In [12]:
filters = {"is_romantic": "yes", "story_type": "sharing a personal story"}

In [29]:
final_df_joined_all_filtered = final_df_joined_all.copy()
for key, value in filters.items():
    final_df_joined_all_filtered = final_df_joined_all_filtered[final_df_joined_all_filtered[key] == value]

In [30]:
final_df_joined_all_filtered.shape

(13805, 36)

In [31]:
def is_age_above_x(value, x=18):
    try:
        age = int(value)
        return age > x
    except:
        return True

In [32]:
final_df_joined_all_filtered["age_female_above_18"] = final_df_joined_all_filtered["age_female"].apply(is_age_above_x)
final_df_joined_all_filtered["age_male_above_18"] = final_df_joined_all_filtered["age_male"].apply(is_age_above_x)
final_df_joined_all_filtered = final_df_joined_all_filtered[(final_df_joined_all_filtered["age_male_above_18"] == True) & (final_df_joined_all_filtered["age_female_above_18"] == True)]
final_df_joined_all_filtered.shape

(13574, 38)

In [33]:
final_df_joined_all_filtered.groupby('subreddit')["post_id"].count().head(12)

subreddit
Infidelity               521
LifeAfterNarcissism     1352
NarcAbuseAndDivorce      698
NarcissisticSpouses     1096
ToxicRelationships       186
abusesurvivors           597
abusiverelationships    2861
domesticviolence        2076
emotionalabuse          1140
relationship_advice      726
relationships            547
survivinginfidelity     1774
Name: post_id, dtype: int64

In [17]:
## sample 5 posts from each subreddit and print post body 
for subreddit in final_df_joined_all_filtered["subreddit"].unique():
    print(subreddit)
    sample = final_df_joined_all_filtered[final_df_joined_all_filtered["subreddit"] == subreddit]["post_body"].sample(5).values
    for post in sample:
        print(post)
        print("-------------------------------------------------")

abusiverelationships
Divorce is going through
Next week. And I just feel so shit about it. I love him but I can’t stay with someone who has hurt me so much. He has had more than enough chances. I’ve waited long enough. I’ve fallen out of love with the potential fantasy of a person who he promised to be. That man I love is in my imagination. I can’t keep disrespecting myself anymore. To stay with someone who just continuously treats you with cruelty is just betraying myself. 

It still hurts to divorce though. I still love that tiny part of him. 10 years. It breaks my heart I know it’s hurting him. I feel for him but I have to choose me. It’s just so hard stepping over and I’m so scared of his reaction too. 

I wish he hadn’t been so abusive but he was and now I’m always scared and worried about his reaction. Even exs never made me feel like that. 

-------------------------------------------------
I feel alone all the time.
Something I wrote in 2021 about my ex: 

We’ve been together 3

In [18]:
final_df_joined_all_filtered["relationship_type"].value_counts()

married                  4304
relationship             4008
dating                   3955
cannot be inferred        757
engaged                   406
other                     264
friends                    58
co-workers                 26
roommates                  14
family                      9
situationship               2
classmates                  1
friends with benefits       1
Name: relationship_type, dtype: int64

In [36]:
final_df_joined_all_filtered.to_csv("/data/home/ilanit.sobol/dv/data/outputs/reddit/labeled/iter2/results_posts_metadata_filtered.csv")

### Load the labeled data

In [1]:
import pandas as pd
import pickle

df_metadata = pd.read_csv("/data/home/ilanit.sobol/dv/data/outputs/reddit/labeled/iter2/results_posts_metadata_filtered.csv")
results_risks = pickle.load(open('/data/home/ilanit.sobol/dv/data/outputs/reddit/labeled/iter2/results_posts_risks.pkl', 'rb'))

In [4]:
df_metadata["subreddit"].unique()

array(['abusiverelationships', 'emotionalabuse', 'NarcissisticSpouses',
       'NarcAbuseAndDivorce', 'domesticviolence', 'abusesurvivors',
       'relationship_advice', 'survivinginfidelity',
       'LifeAfterNarcissism', 'ToxicRelationships', 'relationships',
       'Infidelity'], dtype=object)

In [ ]:
results_risks = pd.DataFrame(results_risks)
results_risks = results_risks[results_risks["output_json"] != None]
expanded_json_df_risks = pd.json_normalize(results_risks['output_json'])
expanded_json_df_risks["post_id"] = results_risks["post_id"]
expanded_json_df_risks = expanded_json_df_risks.set_index("post_id")
results_risks = results_risks.set_index("post_id")
results_df_joined_risks = results_risks.join(expanded_json_df_risks, how='inner')
results_df_joined_risks = results_df_joined_risks.drop(["output_json"], axis=1)
results_df_joined_risks["post_body"] = results_df_joined_risks["story"]

In [23]:
results_df_joined_risks.columns

Index(['story', 'emotional_violence', 'physical_violence', 'sexual_violence',
       'economic_violence', 'past_offenses', 'social_isolation',
       'refuses_treatment', 'suicidal_threats', 'mental_condition',
       'daily_activity_control', 'violent_behavior', 'unemployment',
       'substance_use', 'obsessiveness', 'jealousy', 'outbursts', 'ptsd',
       'hard_childhood', 'emotional_dependency', 'prevention_of_care',
       'fear_based_relationship', 'humiliation', 'physical_threats',
       'presence_of_others_in_assault', 'signs_of_injury', 'property_damage',
       'access_to_weapons', 'gaslighting', 'victim_support_network',
       'offender_support_network', 'attempts_to_end_relationship',
       'refusal_to_end_relationship', 'public_private_discrepancy',
       'narcissistic_traits', 'aggressive_behavior', 'age_victim',
       'age_offender', 'gender_victim', 'gender_offender',
       'spiritual_violence', 'post_body'],
      dtype='object')

In [10]:
df_metadata_risks = df_metadata.merge(results_df_joined_risks, left_on = "post_id", right_on="post_id", how='inner')
df_metadata_risks = df_metadata_risks.reset_index()
df_metadata_risks  = df_metadata_risks.drop_duplicates(subset=['post_body_x'])
df_metadata_risks.shape

(9447, 82)

In [11]:
df_metadata_risks.to_csv("/data/home/ilanit.sobol/dv/data/outputs/reddit/labeled/iter2/results_posts_risks_joined_all.csv")

In [7]:
df_metadata_risks.head()

,Unnamed: 0,post_id,title,post_author,body,date_time,url,score,num_comments,flairs,...,refusal_to_end_relationship,public_private_discrepancy,narcissistic_traits,aggressive_behavior,age_victim,age_offender,gender_victim,gender_offender,spiritual_violence,post_body_y
0,0,1b755nn,I (F38) have just broke up with bf (M44) over ...,pussy_exclusive,Thank you for your advice it gave me strength ...,05/03/24 13:26:38,https://www.reddit.com/r/abusiverelationships/...,10,6,[],...,plausibly,cannot be inferred,cannot be inferred,plausibly,NaN,NaN,NaN,NaN,NaN,I (F38) have just broke up with bf (M44) over ...
1,1,147v73l,Hard time bouncing back,Electrical-Plane-373,I (f38) am having a hard time getting past the...,12/06/23 19:20:53,https://www.reddit.com/r/emotionalabuse/commen...,11,2,[],...,cannot be inferred,cannot be inferred,plausibly,yes,NaN,NaN,NaN,NaN,NaN,Hard time bouncing back\nI (f38) am having a h...
2,2,15eemjv,Their romantic past,Federal-Meal-2513,I (F38) have been with my partner (M41) for al...,31/07/23 12:18:45,https://www.reddit.com/r/NarcissisticSpouses/c...,5,4,[],...,cannot be inferred,plausibly,plausibly,cannot be inferred,NaN,NaN,NaN,NaN,NaN,Their romantic past\nI (F38) have been with my...
3,4,hcyvco,Facebook Facepalm,Katbag,I have a nex constantly stalking my Facebook a...,21/06/20 02:18:40,https://www.reddit.com/r/NarcAbuseAndDivorce/c...,40,47,[],...,plausibly,cannot be inferred,plausibly,cannot be inferred,NaN,NaN,female,male,cannot be inferred,Facebook Facepalm\nI have a nex constantly sta...
4,5,17m9pnb,"My abuser was an extremely socially conscious,...",OneMidnight121,This is one of the aspects of my abuse that ha...,02/11/23 18:08:45,https://www.reddit.com/r/abusiverelationships/...,133,76,[],...,cannot be inferred,yes,plausibly,plausibly,NaN,NaN,male,female,cannot be inferred,"My abuser was an extremely socially conscious,..."


In [8]:
df_metadata_risks["living_with"].value_counts()

no                    4682
cannot be inferred    2636
yes                   2164
Name: living_with, dtype: int64

In [94]:
df_metadata_risks["abusive_relationship"].value_counts()

yes                   7435
cannot be inferred    1201
no                     555
irrelevant             291
Name: abusive_relationship, dtype: int64

In [95]:
df_metadata_risks["physical_threats"].value_counts()

cannot be inferred    7138
plausibly              855
yes                    845
no                     644
Name: physical_threats, dtype: int64